In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
#geoapify_key = "e544d1a5aa944d8fb0b552d745a04636"


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_weather_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,talnakh,69.4865,88.3972,-26.00,92,100,3.53,RU,Sat Feb 17 10:13:25 2024
1,edinburgh of the seven seas,-37.0676,-12.3116,17.46,82,98,6.68,SH,Sat Feb 17 10:13:26 2024
2,bethel,41.3712,-73.4140,0.81,77,100,3.09,US,Sat Feb 17 10:13:26 2024
3,heroica matamoros,25.8833,-97.5000,14.95,96,100,8.23,MX,Sat Feb 17 10:13:13 2024
4,avarua,-21.2078,-159.7750,22.03,94,20,0.51,CK,Sat Feb 17 10:13:30 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
import pandas as pd
import hvplot.pandas  # Make sure hvplot is installed

# Assuming city_data_df is your DataFrame and it has columns 'City', 'Lat', 'Lng', 'Humidity'
# Make sure your DataFrame is loaded correctly
city_data_df = pd.read_csv('output_data/city_weather_data.csv')  # Uncomment and modify if needed

# Create a map using hvplot with points sized by humidity
# Adjust the size factor in the `size` argument as needed to scale the point sizes appropriately
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=city_data_df['Humidity']*0.1,  # Scale size by humidity
                                      hover_cols=['City'], color='Humidity', cmap='blues', alpha=0.6,
                                      tiles='OSM', frame_width=900, frame_height=600)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0) ]
    

filtered_cities_df = filtered_cities_df.dropna()
print(filtered_cities_df)
filtered_cities_df


                  City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
8               saipan  15.1355  145.7010     26.39        89           0   
233             howrah  22.5892   88.3103     22.00        73           0   
239           celestun  20.8667  -90.4000     26.38        64           0   
262              pisco -13.7000  -76.2167     26.12        73           0   
304             monywa  22.1167   95.1333     23.72        33           0   
345  san jose del cabo  23.0589 -109.6977     24.32        39           0   
358     cabo san lucas  22.8909 -109.9124     22.51        49           0   
502            dolores -36.3132  -57.6792     26.77        54           0   
550             dwarka  22.2394   68.9678     23.66        59           0   

     Wind Speed Country                      Date  
8          2.57      MP  Sat Feb 17 10:13:31 2024  
233        0.00      IN  Sat Feb 17 10:14:33 2024  
239        3.47      MX  Sat Feb 17 10:14:34 2024  
262        4.12      

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,saipan,15.1355,145.7010,26.39,89,0,2.57,MP,Sat Feb 17 10:13:31 2024
233,howrah,22.5892,88.3103,22.00,73,0,0.00,IN,Sat Feb 17 10:14:33 2024
239,celestun,20.8667,-90.4000,26.38,64,0,3.47,MX,Sat Feb 17 10:14:34 2024
262,pisco,-13.7000,-76.2167,26.12,73,0,4.12,PE,Sat Feb 17 10:14:37 2024
304,monywa,22.1167,95.1333,23.72,33,0,0.62,MM,Sat Feb 17 10:14:44 2024
345,san jose del cabo,23.0589,-109.6977,24.32,39,0,1.54,MX,Sat Feb 17 10:14:50 2024
358,cabo san lucas,22.8909,-109.9124,22.51,49,0,1.54,MX,Sat Feb 17 10:13:44 2024
502,dolores,-36.3132,-57.6792,26.77,54,0,4.46,AR,Sat Feb 17 10:15:21 2024
550,dwarka,22.2394,68.9678,23.66,59,0,4.49,IN,Sat Feb 17 10:15:41 2024


Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Selecting specific columns to copy into the new DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Optionally, you can add an empty column for hotel information if you plan to fill it later
hotel_df['Hotel Name'] = ' '
# Display the first few rows of the new DataFrame to confirm its structure
print(hotel_df.head())
hotel_df.head()

         City Country      Lat       Lng  Humidity Hotel Name
8      saipan      MP  15.1355  145.7010        89           
233    howrah      IN  22.5892   88.3103        73           
239  celestun      MX  20.8667  -90.4000        64           
262     pisco      PE -13.7000  -76.2167        73           
304    monywa      MM  22.1167   95.1333        33           


,City,Country,Lat,Lng,Humidity,Hotel Name
8,saipan,MP,15.1355,145.7010,89,
233,howrah,IN,22.5892,88.3103,73,
239,celestun,MX,20.8667,-90.4000,64,
262,pisco,PE,-13.7000,-76.2167,73,
304,monywa,MM,22.1167,95.1333,33,


For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
import requests
import pandas as pd

# Assuming hotel_df is defined with columns ['City', 'Country', 'Lat', 'Lng']

# Set the search radius, in meters
radius = 10000  # 10,000 meters

# Set base URL for Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Define API parameters for each city
    params = {
        "lat": row['Lat'],
        "lon": row['Lng'],
        "filter": f"radius:{radius}",
        "type": "accommodation",
        "limit": 1,  # Only fetch the first result
        "apiKey": "e544d1a5aa944d8fb0b552d745a04636"  # Replace with your actual Geoapify API key
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract the hotel name from the first result, if available
        if data['features']:
            hotel_name = data['features'][0]['properties']['name']
            hotel_df.loc[index, "Hotel Name"] = hotel_name
            print(f"{row['City']} - nearest hotel: {hotel_name}")
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            print(f"{row['City']} - No hotel found")
    else:
        print(f"Failed to fetch data for {row['City']}, status code: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

# Display a sample of the DataFrame to verify the results
print(hotel_df.head())



Starting hotel search
Failed to fetch data for saipan, status code: 400
Failed to fetch data for howrah, status code: 400
Failed to fetch data for celestun, status code: 400
Failed to fetch data for pisco, status code: 400
Failed to fetch data for monywa, status code: 400
Failed to fetch data for san jose del cabo, status code: 400
Failed to fetch data for cabo san lucas, status code: 400
Failed to fetch data for dolores, status code: 400
Failed to fetch data for dwarka, status code: 400
         City Country      Lat       Lng  Humidity          Hotel Name
8      saipan      MP  15.1355  145.7010        89  API request failed
233    howrah      IN  22.5892   88.3103        73  API request failed
239  celestun      MX  20.8667  -90.4000        64  API request failed
262     pisco      PE -13.7000  -76.2167        73  API request failed
304    monywa      MM  22.1167   95.1333        33  API request failed


In [7]:
# Set the search radius in meters (e.g., 10,000 meters for 10 kilometers)
radius = 10000

params = {
    "type": "accommodation",
    "filter": "category:lodging",
    "limit": 1,  # Fetch only the first result
    "apiKey": "e544d1a5aa944d8fb0b552d745a04636"  # Replace with your actual Geoapify API key
}


In [8]:
import requests

base_url = "https://api.geoapify.com/v2/places/search"

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # Ensure the correct format and values for the parameters
    params = {
        "lat": row['Lat'],
        "lon": row['Lng'],
        "filter": "category:lodging",
        "limit": 1,  # Fetch only the first result
        "apiKey": "e544d1a5aa944d8fb0b552d745a04636"  # Use your actual Geoapify API key
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract the hotel name from the first result, if available
        if data['features']:
            hotel_name = data['features'][0]['properties']['name']
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"Failed to fetch data for {row['City']}, status code: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

print(hotel_df.head())


Starting hotel search
Failed to fetch data for saipan, status code: 404
Failed to fetch data for howrah, status code: 404
Failed to fetch data for celestun, status code: 404
Failed to fetch data for pisco, status code: 404
Failed to fetch data for monywa, status code: 404
Failed to fetch data for san jose del cabo, status code: 404
Failed to fetch data for cabo san lucas, status code: 404
Failed to fetch data for dolores, status code: 404
Failed to fetch data for dwarka, status code: 404
         City Country      Lat       Lng  Humidity          Hotel Name
8      saipan      MP  15.1355  145.7010        89  API request failed
233    howrah      IN  22.5892   88.3103        73  API request failed
239  celestun      MX  20.8667  -90.4000        64  API request failed
262     pisco      PE -13.7000  -76.2167        73  API request failed
304    monywa      MM  22.1167   95.1333        33  API request failed


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

import hvplot.pandas  # for plotting with hvplot
import geopandas as gpd  # for geographical operations

# Assuming hotel_df is your DataFrame and it already contains 'Lat', 'Lng', 'City', 'Country', and 'Hotel Name' columns

# Convert your DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))

# Define columns to display on hover
hover_cols = ['City', 'Country', 'Hotel Name']

# Plot using hvplot with hover information
map_plot = gdf.hvplot(
    geo=True, 
    hover_cols=hover_cols, 
    tiles='OSM', 
    color='Hotel Name',  # This will use different colors for different hotels
    cmap='Category20',  # Color map for distinguishing hotels
    alpha=0.5,  # Transparency of the points
    size=200,  # Size of the points
    frame_width=700, 
    frame_height=500,
    title='Hotels Map'
)

# Enable hover tool
map_plot.opts(tools=['hover'])

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Hotel Name,City,Country)